In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import cross_val_score, RandomizedSearchCV, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer

In [2]:
# Import the presplit data
X_train = pd.read_csv('data/X_train.csv')
X_test = pd.read_csv('data/X_test.csv')
y_train = pd.read_csv('data/y_train.csv')
y_test = pd.read_csv('data/y_test.csv')

In [3]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40095 entries, 0 to 40094
Data columns (total 32 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     40095 non-null  int64  
 1   amount_tsh             40095 non-null  float64
 2   funder                 40095 non-null  object 
 3   gps_height             40095 non-null  int64  
 4   installer              40095 non-null  object 
 5   longitude              40095 non-null  float64
 6   latitude               40095 non-null  float64
 7   num_private            40095 non-null  int64  
 8   basin                  40095 non-null  object 
 9   region                 40095 non-null  object 
 10  region_code            40095 non-null  int64  
 11  district_code          40095 non-null  int64  
 12  population             40095 non-null  int64  
 13  public_meeting         40095 non-null  object 
 14  scheme_management      40095 non-null  object 
 15  pe

In [4]:
X_train.drop('id', axis = 1, inplace=True)
X_test.drop('id', axis = 1, inplace=True)

In [5]:
# X_train = pd.get_dummies(X_train, drop_first=True)

# X_test = pd.get_dummies(X_test, drop_first=True)

In [6]:
y_train.drop('id', inplace=True, axis = 1)
y_test.drop('id', axis = 1, inplace=True)

In [7]:
y_train = np.array(y_train)
y_train = y_train.reshape(40095, )

In [8]:
y_test = np.array(y_test)
y_test = y_test.reshape(13365, )

In [9]:
rfc = RandomForestClassifier()
ss = StandardScaler()

X_train_nums = X_train.select_dtypes(exclude=object)
X_test_nums = X_test.select_dtypes(exclude=object)

X_train_nums_scaled = ss.fit_transform(X_train_nums)
X_test_nums_scaled = ss.transform(X_test_nums)

In [10]:
rfc.fit(X_train_nums_scaled, y_train)

preds = rfc.predict(X_test_nums_scaled)

In [11]:
train_scores = rfc.score(X_train_nums_scaled, y_train)
test_scores = rfc.score(X_test_nums_scaled, y_test)
print('The Mode\'s accuracy on the training data is', round(train_scores, 4))
print('The Model\'s accuracy on the test data is', round(test_scores, 4))

The Mode's accuracy on the training data is 0.9858
The Model's accuracy on the test data is 0.7166


In [12]:
log_loss_rfc = cross_val_score(rfc, X_train_nums_scaled, y_train, scoring='neg_log_loss', n_jobs = -1)
log_loss_rfc = -log_loss_rfc.mean()
print('Log Loss:', log_loss_rfc)

Log Loss: 1.0042233873961206


This inital model that only contains numerical data is severly overfitting the data. The next model will need to tune some hyperparameters to reduce the gap between the training and testing data. Our log loss has been decreased from the dummy classifer, but there is still room for improvement

In [13]:
pipline_1 = Pipeline([('ss', StandardScaler()), ('rfc', RandomForestClassifier())])

In [14]:
pipline_1.get_params()

{'memory': None,
 'steps': [('ss', StandardScaler()), ('rfc', RandomForestClassifier())],
 'verbose': False,
 'ss': StandardScaler(),
 'rfc': RandomForestClassifier(),
 'ss__copy': True,
 'ss__with_mean': True,
 'ss__with_std': True,
 'rfc__bootstrap': True,
 'rfc__ccp_alpha': 0.0,
 'rfc__class_weight': None,
 'rfc__criterion': 'gini',
 'rfc__max_depth': None,
 'rfc__max_features': 'auto',
 'rfc__max_leaf_nodes': None,
 'rfc__max_samples': None,
 'rfc__min_impurity_decrease': 0.0,
 'rfc__min_impurity_split': None,
 'rfc__min_samples_leaf': 1,
 'rfc__min_samples_split': 2,
 'rfc__min_weight_fraction_leaf': 0.0,
 'rfc__n_estimators': 100,
 'rfc__n_jobs': None,
 'rfc__oob_score': False,
 'rfc__random_state': None,
 'rfc__verbose': 0,
 'rfc__warm_start': False}

In [15]:
params_1 = {
    'rfc__max_depth': list(range(5,50,5)),
    'rfc__min_samples_leaf': list(range(5,50,5)),
    'rfc__n_estimators': list(range(25,500,25)),
}

In [16]:
rs_1 = RandomizedSearchCV(pipline_1, params_1, n_jobs=-1, verbose=2, n_iter=50)

rs_1.fit(X_train_nums, y_train)

Fitting 5 folds for each of 50 candidates, totalling 250 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:  1.7min
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:  8.1min
[Parallel(n_jobs=-1)]: Done 250 out of 250 | elapsed: 14.8min finished


RandomizedSearchCV(estimator=Pipeline(steps=[('ss', StandardScaler()),
                                             ('rfc',
                                              RandomForestClassifier())]),
                   n_iter=50, n_jobs=-1,
                   param_distributions={'rfc__max_depth': [5, 10, 15, 20, 25,
                                                           30, 35, 40, 45],
                                        'rfc__min_samples_leaf': [5, 10, 15, 20,
                                                                  25, 30, 35,
                                                                  40, 45],
                                        'rfc__n_estimators': [25, 50, 75, 100,
                                                              125, 150, 175,
                                                              200, 225, 250,
                                                              275, 300, 325,
                                                              

In [17]:
rs_1.score(X_train_nums, y_train), rs_1.score(X_test_nums, y_test)

(0.8274847237810201, 0.7215114104002993)

In [18]:
rs_1.best_params_

{'rfc__n_estimators': 350, 'rfc__min_samples_leaf': 5, 'rfc__max_depth': 40}

In [19]:
x = False
while x == True:
    assert print('Need to fix the imbalance between the training data and the testing data')
    assert print('Fix below')

In [66]:
X_train_nums = X_train.select_dtypes(exclude = object)

X_train_cat = X_train.select_dtypes('object')

In [67]:
numerical_pipeline = Pipeline(steps=[
    ('ss', StandardScaler())
])
                
categorical_pipeline = Pipeline(steps=[
    ('ohe', OneHotEncoder(drop='first', handle_unknown = 'ignore'))
])


In [68]:
ohe = OneHotEncoder(handle_unknown='ignore')

ohe.fit(X_train)

OneHotEncoder(handle_unknown='ignore')

In [69]:
trans = ColumnTransformer(transformers=[
    ('numerical', numerical_pipeline, X_train_nums.columns),
    ('categorical', categorical_pipeline, X_train_cat.columns)
], verbose=True, n_jobs=-1)

In [70]:
model_pipe = Pipeline(steps=[
    ('trans', trans),
    ('rfr', RandomForestClassifier(verbose=1, n_jobs=-1))
])

In [71]:
model_pipe.fit(X_train, y_train)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   10.0s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:   27.7s finished


Pipeline(steps=[('trans',
                 ColumnTransformer(n_jobs=-1,
                                   transformers=[('numerical',
                                                  Pipeline(steps=[('ss',
                                                                   StandardScaler())]),
                                                  Index(['amount_tsh', 'gps_height', 'longitude', 'latitude', 'num_private',
       'region_code', 'district_code', 'population', 'construction_year',
       'funder_A/co Germany',
       ...
       'waterpoint_type_communal standpipe multiple', 'waterpoint_type_dam',
       'wate...
       'waterpoint_type_other', 'waterpoint_type_group_communal standpipe',
       'waterpoint_type_group_dam', 'waterpoint_type_group_hand pump',
       'waterpoint_type_group_improved spring', 'waterpoint_type_group_other'],
      dtype='object', length=3520)),
                                                 ('categorical',
                                          

In [72]:
model_pipe.score(X_train, y_train)

[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.2s
[Parallel(n_jobs=8)]: Done 100 out of 100 | elapsed:    0.6s finished


0.9973063973063973

In [73]:
model_pipe.score(X_test, y_test)

ValueError: Number of features of the input must be equal to or greater than that of the fitted transformer. Transformer n_features is 3520 and input n_features is 2046.

In [ ]:
model_pipe.predict_proba(X_train)

In [74]:
cv_1 = cross_val_score(model_pipe, X_train, y_train, scoring='neg_log_loss', n_jobs = -1)

In [75]:
-cv_1

array([0.89672136, 0.84950494, 0.88854785, 0.83429731, 0.85628461])

In [76]:
cv_1 = cross_val_score(model_pipe, X_test, y_test, scoring='neg_log_loss', n_jobs = -1)
-cv_1

array([nan, nan, nan, nan, nan])